In [1]:
import sys
sys.path.append("../src")
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Activation, Dropout, Lambda, Dense
from tensorflow.keras import Sequential
import os
import pandas as pd
from util import *
from model import *
import json

In [2]:
with open("../config/age_parameters.json") as param:
    parameters = json.load(param)["load_data"]
param.close()

train_label = parameters["train_label_path"]
train_image = parameters["train_image_path"]
resize = parameters["resize"]
TRAIN_TEST_SPLIT = parameters["TRAIN_TEST_SPLIT"]


In [3]:
data_generator = make_generator(train_label, train_image, resize, TRAIN_TEST_SPLIT)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()

In [ ]:
test_batch_size = 128
model = OutputModel().assemble_full_model(resize, resize)
model.load_weights("../models/age_model_1/cp-0037.ckpt")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
age_pred = model.predict_generator(test_generator, steps=len(test_idx)//test_batch_size)

In [23]:
test_generator = data_generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true = [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    images.extend(image)
    age_true.extend(labels[0])

age_true = np.array(age_true)


age_true, age_pred = age_true.argmax(axis=-1), age_pred.argmax(axis=-1)

In [27]:
(age_true == age_pred).mean()

0.5142010467980296

In [29]:
from sklearn.metrics import classification_report

cr_race = classification_report(age_true, age_pred, target_names=data_generator.dataset_dict['age_alias'].keys())
print(cr_race)

              precision    recall  f1-score   support

         0-2       0.60      0.44      0.51       562
         3-9       0.67      0.80      0.73      3110
       10-19       0.56      0.24      0.33      2727
       20-29       0.51      0.76      0.61      7614
       30-39       0.45      0.36      0.40      5848
       40-49       0.43      0.29      0.34      3200
       50-59       0.43      0.48      0.45      1861
       60-69       0.44      0.24      0.31       817
more than 70       0.65      0.26      0.37       245

    accuracy                           0.51     25984
   macro avg       0.53      0.43      0.45     25984
weighted avg       0.51      0.51      0.49     25984

